# Fine Tuning Tiny Yolo

The Tiny Yolo Network is fine tuned to only detect people.

## Prepare Workspace

### Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


### Create the directories needed and place uploaded files inside them

In [ ]:
!pip install torchinfo
!pip install torchvision pillow

!mkdir /content/data

!cp /content/drive/MyDrive/eml_challenge/data/person_indices.json /content/data
!cp -r /content/drive/MyDrive/eml_challenge/utils /content
!cp /content/drive/MyDrive/eml_challenge/tinyyolov2.py /content

### Append directory paths to system path

In [ ]:
import sys
sys.path.append('/content')
sys.path.append('/content/data')
sys.path.append('/content/utils')
sys.path.append('/content/drive/MyDrive/eml_challenge/weights')

## Executing Workspace

### Importing essential libraries

In [ ]:
import torch
import torchinfo
import torch.nn as nn
import numpy as np

# A subset of VOCDataLoader just for one class (person) (0)
from utils.dataloader import VOCDataLoaderPerson
# Import ReduceLROnPlateau to reduce learning rate of optimizer after Plateau
from torch.optim.lr_scheduler import ReduceLROnPlateau

data_loader = VOCDataLoaderPerson(train=True, batch_size=128, shuffle=True)
test_loader = VOCDataLoaderPerson(train=False, batch_size=1)

from tinyyolov2 import TinyYoloV2
from utils.loss import YoloLoss
import tqdm
from utils.ap import precision_recall_levels, ap, display_roc
from utils.yolo import nms, filter_boxes
from utils.viz import display_result

### Define Early Stopping class

In [ ]:
class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0,
                 path='/content/drive/MyDrive/eml_challenge/weights/checkpoint.pt'):
        """
        Args:
            patience (int): How long to wait after last improvement.
            verbose (bool): If True, prints a message for each validation metric improvement.
            delta (float): Minimum change in the monitored metric to qualify as an improvement.
            path (str): Path to save the best model checkpoint.
        """
        self.patience = patience
        self.verbose = verbose
        self.delta = delta
        self.path = path
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.avg_precision_min = 0 # Track the minimum average precision

    def __call__(self, avg_precision, model):
        score = avg_precision  # Positiv because we maximize AP

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(avg_precision, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f"EarlyStopping counter: {self.counter} out of {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(avg_precision, model)
            self.counter = 0

    def save_checkpoint(self, avg_precision, model):
        """Save model when average precision increases."""
        if self.verbose:
            print(f"Average Precision increased ({self.avg_precision_min:.6f} --> {avg_precision:.6f}). Saving model...")
        torch.save(model.state_dict(), self.path)
        self.avg_precision_min = avg_precision


### Fine Tuning function definition

In [ ]:
def fine_tune(net: nn.Module, sd,
              data_loader: torch.utils.data.DataLoader, test_loader: torch.utils.data.DataLoader,
              num_test_samples: int=0):

    if torch.cuda.is_available():
      torch_device = torch.device("cuda")
      print("Using GPU")
    else:
      torch_device = torch.device("cpu")
      print("Using CPU")

    test_AP = []
    test_precision = []
    test_recall = []

    #We load all parameters from the pretrained dict except for the last layer
    # net.load_state_dict({k: v for k, v in sd.items() if not '9' in k}, strict=False)
    net.load_state_dict({k: v for k, v in sd.items() if not '9' in k}, strict=False)
    net.eval()
    # Move weights to device
    net.to(torch_device)

    #We only train the last layer (conv9)
    for key, param in net.named_parameters():
        if any(x in key for x in ['1', '2', '3', '4', '5', '6', '7']):
            param.requires_grad = False

    # Definition of the loss
    criterion = YoloLoss(anchors=net.anchors)
    # Definition of the optimizer
    learning_rate = 0.001
    optimizer = torch.optim.Adam(filter(lambda x: x.requires_grad, net.parameters()), lr=learning_rate)
    # Define the ReduceLROnPlateau scheduler
    scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=5, verbose=True)
    # Initialize EarlyStopping
    early_stopping = EarlyStopping(patience=7, verbose=True)

    for epoch in range(NUM_EPOCHS):
        print(f"Epoch: {epoch}")
        net.train()
        if epoch != 0:
            for idx, (input, target) in tqdm.tqdm(enumerate(data_loader), total=len(data_loader)):
                optimizer.zero_grad()
                # Move Inputs and targets to Device
                input  = input.to(torch_device)
                target = target.to(torch_device)
                #Yolo head is implemented in the loss for training, therefore yolo=False
                output = net(input, yolo=False)
                loss, _ = criterion(output, target)
                loss.backward()
                optimizer.step()

        with torch.no_grad():
            for idx, (input, target) in tqdm.tqdm(enumerate(test_loader), total=num_test_samples):
                net.eval()
                input  = input.to(torch_device)
                target = target.to(torch_device)
                output = net(input, yolo=True)
                #The right threshold values can be adjusted for the target application
                output = filter_boxes(output, 0.0)
                output = nms(output, 0.5)
                # Calculate precision and recall for each sample
                precision, recall = precision_recall_levels(target[0], output[0])
                test_precision.append(precision)
                test_recall.append(recall)
                if idx == num_test_samples:
                    break

        #Calculation of average precision with collected samples
        average_precision = ap(test_precision, test_recall)
        test_AP.append(average_precision)
        #plot ROC
        display_roc(test_precision, test_recall)
        print('average precision', test_AP)
        # Adjust learning rate in case of a Plateau of AP
        scheduler.step(average_precision)
        print(f"learning rate: {scheduler.get_last_lr()}")
        # Stop training in case there is no further improvement of AP
        early_stopping(average_precision, net)
        if early_stopping.early_stop:
            print("Early stopping triggered. Stopping training.")
            torch.cuda.empty_cache()
            return net.state_dict()

    torch.cuda.empty_cache()

    return net.state_dict()

## Testing Workspace

In [ ]:
# We define a tinyyolo network with only two possible classes
net = TinyYoloV2(num_classes=1)
sd = torch.load("/content/drive/MyDrive/eml_challenge/weights/voc_pretrained.pt")

# Number of Epochs
NUM_EPOCHS = 50

sd_fine_tuned = fine_tune(net, sd, data_loader, test_loader, num_test_samples=350)
torch.save(sd_fine_tuned, '/content/drive/MyDrive/eml_challenge/weights/voc_fine_tuned.pt')